In [13]:
from pathlib import Path
from decode_fish.funcs.utils import *
import glob

base_string = f'bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py'   

def make_sweep_str(pfile, pars, group):
    
    base_str = base_string + f' +experiment={pfile}'
    name = ''
    
    for k in pars:
        base_str += ' ' + k + f'={pars[k]}'
        name += k.split('.')[-1]+f':{pars[k]}x'
    name = name[:-1]
    
    return base_str + f' run_name={name} output.group={group}' + '\n'
    

def make_named_str(pfile, name, group):
    
    base_str = base_string + f' +experiment={pfile}'
    
    return base_str + f' run_name={name} output.group={group}' + '\n'

In [22]:
group = 'sweep_m3'
pfile = 'Dlg1_m3'

variable_col = param_iter()
variable_col.add('noise.theta', 29.41*1.5)
variable_col.add('bg_estimation.fractal.scale', 2000, 3000)
variable_col.add('bg_estimation.fractal.persistence', 0.5,0.75)
variable_col.add('prob_generator.high', 0.0003, 0.00015)
par_prod = variable_col.param_product()

In [31]:
# Path.mkdir(model_path/group, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(pfile, pars, group)
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=fig_sim_m7 training.mic.enabled=True training.int.enabled=True run_name=enabled:Truexenabled:True output.group=sweep_fig7

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=fig_sim_m7 training.mic.enabled=True training.int.enabled=False run_name=enabled:Truexenabled:False output.group=sweep_fig7

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=fig_sim_m7 training.mic.enabled=False 

In [30]:
group = 'sweep_fig7'
pfile = 'fig_sim_m7'

variable_col = param_iter()
variable_col.add('training.mic.enabled', True, False)
variable_col.add('training.int.enabled', True, False)
# variable_col.add('training.mic.opt._target_', 'torch_optimizer.AdaBound', 'torch_optimizer.AdaMod', 'torch_optimizer.Adafactor', 'torch_optimizer.AdamP', 'torch_optimizer.AggMo', 'torch_optimizer.DiffGrad', 
#                 'torch_optimizer.NovoGrad', 'torch_optimizer.PID', 'torch_optimizer.QHAdam', 'torch_optimizer.QHM', 'torch_optimizer.RAdam', 'torch_optimizer.SGDP', 'torch_optimizer.SGDW', 'torch_optimizer.Shampoo',
#                 'torch_optimizer.SWATS', 'torch_optimizer.YOGI')

par_prod = variable_col.param_product()

In [14]:
from decode_fish.imports import *

In [29]:
exps = ['msp300', 'N2_352', 'rab11', 'scrib', 'sgg', 'Dlg1']
# exps = [ 'scrib', 'Dlg1']
ver = 'f1'
name = 'f1_none'

with open('run_slurm.sh', 'w') as f:
    for exp in exps:  
        cfg = OmegaConf.load(f'config/experiment/{exp}_{ver}.yaml')
#         exec_str = base_string + f' +experiment={exp}_{ver}  run_name={exp}_{name}' + '\n'
        exec_str = base_string + f' +experiment={exp}_{ver}  run_name={exp}_{name} noise.theta={cfg.noise.theta*1.5} microscope.norm=none' + '\n'
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=msp300_f1  run_name=msp300_f1_none noise.theta=54.344570027378 microscope.norm=none

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=N2_352_f1  run_name=N2_352_f1_none noise.theta=23.27232633537633 microscope.norm=none

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=rab11_f1  run_name=rab11_f1_none noise.theta=81.16765723381953 microscope.norm=none

bsub -gpu "num=1" -q gpu_any 

In [ ]:
# bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/sim_eval.py sweep_name=fig2

In [ ]:
exps = ['msp300', 'N2_352', 'rab11', 'scrib', 'sgg', 'Dlg1']
# exps = [ 'scrib', 'Dlg1']
ver = 'f1'
name = 'f1_none'